# Construction du modèle de reconnaissance de caractère basé sur le dataset MIST

Dans ce notebook sera présenté la création du modèle qui permettra de **classifier** les lettres de l'alphabet de a-z et A-Z.

Pour ce faire nous allons utiliser le dataset d'images prétraités créés par le notebook [NIST-preprocessing](/notebooks/notebooks/character_recognition/NIST-preprocessing.ipynb)

Pour exécuter ce notebook, veillez à ce que le jeu de données soit bien sous la forme suivante.
- data
    - processed
         - NIST-dataset
            - train
                - a000001.png
                - a000002.png
                ...
                ...
                - a00000n.png
            - test_set
                - a000001.png
                - a000002.png
                ...
                ...
                - a00000n.png


## Importation des dépendances

In [ ]:
import os
from random import shuffle
from string import ascii_lowercase, ascii_uppercase

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from tensorflow import keras
from tensorflow.keras import layers

2023-03-02 15:29:12.786181: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 15:29:12.911723: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/alexandre/Programmes/Python-3.6.4/
2023-03-02 15:29:12.911740: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-02 15:29:13.677172: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: can

## Définition des paramètres du notebook

In [ ]:
NIST_PROCESSED_PATH = "../../data/processed/NIST-dataset"
TRAIN_SET_PATH = os.path.join(NIST_PROCESSED_PATH, "train")
TEST_SET_PATH = os.path.join(NIST_PROCESSED_PATH, "test")

## Importation du dataset

Dans cette section, nous allons charger le dataset afin d'obtenir un train_data_set, train_label_set, ainsi qu'un test_data_set et un test_label_set

### Chargement du dataset depuis le disque

Déclaration de la fonction qui va charger l'image en récupérant le label associé à celle-ci

In [ ]:
def load_img_to_np_array(file_path):
    full_name = os.path.basename(file_path)
    file_name = os.path.splitext(full_name)[0]
    label = file_name[0]

    return label, plt.imread(file_path)

Cette fonction vérifie si un fichier est bien un png.

In [ ]:
def is_png(file_name):
    extension = file_name.split(".")[1]
    if extension == "png":
        return True
    return False

file_name1 = "AOO1.png"
file_name2 = "AOO1.jpeg"

print(f"{file_name1} est un png ? {is_png(file_name1)}")
print(f"{file_name2} est un png ? {is_png(file_name2)}")

AOO1.png est un png ? True
AOO1.jpeg est un png ? False


On récupère tous les fichiers .png dans un repertoire spécifié en paramètre (dir_path)


In [ ]:
def select_png_from_dir(dir_path):
    files_in_dir = []
    with os.scandir(dir_path) as entries:
        for entry in entries:
            file_name = entry.name
            if entry.is_file() and is_png(file_name):
                file_path = os.path.join(dir_path, file_name)
                files_in_dir.append(file_path)
    return files_in_dir

On récupère tous les fichiers en extension png dans les répertoires train et test

In [ ]:
train_img_list = select_png_from_dir(TRAIN_SET_PATH)
test_img_list = select_png_from_dir(TEST_SET_PATH)
train_img_list[0:5]

['../../data/processed/NIST-dataset/train/A000.png',
 '../../data/processed/NIST-dataset/train/a000.png',
 '../../data/processed/NIST-dataset/train/A001.png',
 '../../data/processed/NIST-dataset/train/a001.png',
 '../../data/processed/NIST-dataset/train/A002.png']

Cette fonction a pour but de créer un data_set sous forme [ (label, image array), ... ]

In [ ]:
def load_data_set(data_img_list):
    data_set = []

    for img_path in data_img_list:
        elem = load_img_to_np_array(img_path)
        data_set.append(elem)

    return data_set

On créer le set d'entrainement et de test

In [ ]:
train_set = load_data_set(train_img_list)
test_set = load_data_set(test_img_list)
test_set[0]

('a',
 array([[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]], dtype=float32))

## Mélange des sets et codage des labels en chiffre

On mélange les sets

In [ ]:
shuffle(train_set)
shuffle(test_set)

On récupère les données et leurs labels dans des listes distinctes

In [ ]:
y_raw_train, x_raw_train = list(zip(*train_set))
y_raw_test, x_raw_test = list(zip(*test_set))
print(y_raw_train[0])
print(x_raw_train[0].shape)

Q
(41, 41)


On récupère tous les labels possibles

In [ ]:
label_list = [char for char in ascii_lowercase + ascii_uppercase]
print(label_list[0:5])

['a', 'b', 'c', 'd', 'e']


On définit l'encoder à utiliser

In [34]:
encoder = LabelBinarizer()
encoder.fit(label_list)

LabelBinarizer()

On encode nos données au format OneShot

In [36]:
y_train = encoder.transform(y_raw_train)
y_test = encoder.transform(y_raw_test)
y_train.shape

(8454, 52)

In [37]:
x_train = np.array(x_raw_train)
x_test = np.array(x_raw_test)
x_train.shape

(8454, 41, 41)

## Création du modèle de classification

In [38]:
model = keras.Sequential([
    keras.Input(shape=(41,41)),
    layers.Flatten(input_shape=(41, 41)),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(52, activation="softmax"),
])

model.compile(optimizer="Adam", loss="mse", metrics=["acc", "mse"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 1681)              0         
                                                                 
 dense_8 (Dense)             (None, 100)               168200    
                                                                 
 dense_9 (Dense)             (None, 100)               10100     
                                                                 
 dense_10 (Dense)            (None, 100)               10100     
                                                                 
 dense_11 (Dense)            (None, 52)                5252      
                                                                 
Total params: 193,652
Trainable params: 193,652
Non-trainable params: 0
_________________________________________________________________


## Entrainement du modèle

In [57]:
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
265/265 [==============================] - 1s 2ms/step - loss: 0.0043 - acc: 0.8506 - mse: 0.0043
Epoch 2/20
265/265 [==============================] - 1s 2ms/step - loss: 0.0039 - acc: 0.8652 - mse: 0.0039
Epoch 3/20
265/265 [==============================] - 1s 2ms/step - loss: 0.0039 - acc: 0.8650 - mse: 0.0039
Epoch 4/20
265/265 [==============================] - 1s 2ms/step - loss: 0.0040 - acc: 0.8620 - mse: 0.0040
Epoch 5/20
265/265 [==============================] - 1s 2ms/step - loss: 0.0041 - acc: 0.8575 - mse: 0.0041
Epoch 6/20
265/265 [==============================] - 1s 2ms/step - loss: 0.0038 - acc: 0.8665 - mse: 0.0038
Epoch 7/20
265/265 [==============================] - 1s 2ms/step - loss: 0.0043 - acc: 0.8508 - mse: 0.0043
Epoch 8/20
265/265 [==============================] - 1s 2ms/step - loss: 0.0036 - acc: 0.8777 - mse: 0.0036
Epoch 9/20
265/265 [==============================] - 1s 2ms/step - loss: 0.0039 - acc: 0.8649 - mse: 0.0039
Epoch 10/20
265/265

## Test du modèle sur le jeu de test

In [58]:
model.evaluate(x_test, y_test, verbose=2)

114/114 - 0s - loss: 0.0108 - acc: 0.6412 - mse: 0.0108 - 138ms/epoch - 1ms/step


[0.010805310681462288, 0.6411813497543335, 0.010805310681462288]